In [40]:
import pandas as pd
import pickle
import re
import numpy as np

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [34]:
def count_obj(tweet, obj):
  return len(re.findall(obj, tweet))  

def contain_obj(tweets, obj):
  count = 0
  for tweet in tweets:
    if obj in tweet:
      count += 1
  return count

In [17]:
results_deep=pd.read_csv('predictions.csv', delimiter=',', encoding='utf-8')

results_trad=pd.read_csv('predictions_svm_stems_other.csv', delimiter=',', encoding='utf-8')

results = results_deep.merge(results_trad, on='id')

results.to_csv('predictions_all.csv', index=False)

In [18]:
results

,id,tweet,true_label,bert_label,gpt_label,trad_label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,NOT,OFF,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,NOT,NOT,NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,NOT,NOT,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT,NOT,NOT,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,NOT,NOT,NOT
...,...,...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,OFF,NOT,NOT,OFF
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT,NOT,NOT,NOT
857,67018,3 people just unfollowed me for talking about ...,OFF,OFF,OFF,OFF
858,50665,#WednesdayWisdom Antifa calls the right fascis...,NOT,OFF,NOT,OFF


In [82]:
all_correct = results.query('true_label == gpt_label == trad_label')
all_correct

,id,tweet,true_label,bert_label,gpt_label,trad_label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,NOT,OFF,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,NOT,NOT,NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,NOT,NOT,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT,NOT,NOT,NOT
5,83681,. . . What the fuck did he do this time?,OFF,OFF,OFF,OFF
...,...,...,...,...,...,...
852,51218,"#BrettKavanaugh ... but anyway, there is the...",NOT,NOT,NOT,NOT
853,41438,All two of them taste like ass. URL,OFF,OFF,OFF,OFF
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT,NOT,NOT,NOT
857,67018,3 people just unfollowed me for talking about ...,OFF,OFF,OFF,OFF


In [83]:
all_wrong = results.query('true_label != gpt_label == trad_label')
all_wrong

,id,tweet,true_label,bert_label,gpt_label,trad_label
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,NOT,NOT,NOT
25,83155,#HIAC Damn Matt Hardy and Randy Orton put on ...,NOT,OFF,OFF,OFF
30,88490,#ChristineBlaseyFord is your #Kavanaugh accuse...,OFF,NOT,NOT,NOT
41,59200,#Georgetown Classmate Says #Left Accuser is Ab...,NOT,NOT,OFF,OFF
58,54053,Are you fucking serious? URL,NOT,OFF,OFF,OFF
...,...,...,...,...,...,...
774,41590,@USER #Rosie makes me nauseous!🤮 URL,OFF,NOT,NOT,NOT
778,14640,#BREXIT deal HAS been reached - and will be un...,OFF,NOT,NOT,NOT
783,88822,@USER @USER Bull crap. You know she doesn't ca...,NOT,OFF,OFF,OFF
826,51762,#auspol I don't know why he is still in his j...,OFF,NOT,NOT,NOT


In [84]:
#deep_disagreement = results.query('bert_label != gpt_label')
#deep_disagreement

In [85]:
# deep true, trad false
deep_true_trad_false = results.query('true_label == gpt_label != trad_label')
#deep_true_trad_false

In [86]:
# deep false, trad true
deep_false_trad_true = results.query('true_label == trad_label != gpt_label')
#deep_false_trad_true

In [87]:
counts, users, hashtags, urls, mean_lengths, stdevs, offs, nots, offs_count, nots_count= [], [], [], [], [], [],[],[],[],[]
for d in [results,all_correct,all_wrong,deep_true_trad_false,deep_false_trad_true]:
  counts.append(len(d))
  users.append(contain_obj(d['tweet'], '@USER')/len(d)) # how many tweets contan @USER
  hashtags.append(contain_obj(d['tweet'], '#')/len(d)) # how many tweets contain #
  urls.append(contain_obj(d['tweet'], 'URL')/len(d)) # how many tweets contain URL
  l = [len(tweet) for tweet in d['tweet']]
  mean_lengths.append(np.mean(l))
  stdevs.append(np.std(l))
  offs.append(len(d[d['true_label']== 'OFF'])/len(d))
  nots.append(len(d[d['true_label']== 'NOT'])/len(d))
  offs_count.append(len(d[d['true_label']== 'OFF']))
  nots_count.append(len(d[d['true_label']== 'NOT']))

In [88]:
name_dict = {
    'name':['test_dataset','all_correct','all_wrong','deep_true_trad_false','deep_false_trad_true'],
    'count': counts,
    'user': users,
    'hashtag':hashtags,
    'url':urls,
    'mean_length':mean_lengths,
    'stdev':stdevs,
    'count_true_off': offs_count,
    'count_true_not': nots_count,
    'percent_true_off': offs,
    'percent_true_not': nots,
}
df_analysis = pd.DataFrame(name_dict)

In [89]:
df_analysis

,name,count,user,hashtag,url,mean_length,stdev,count_true_off,count_true_not,percent_true_off,percent_true_not
0,test_dataset,860,0.384884,0.737209,0.560465,146.160465,78.019198,240,620,0.279070,0.720930
1,all_correct,631,0.372425,0.735341,0.567353,141.323296,77.468301,135,496,0.213946,0.786054
2,all_wrong,79,0.417722,0.734177,0.493671,159.177215,82.962056,49,30,0.620253,0.379747
3,deep_true_trad_false,98,0.448980,0.714286,0.561224,164.653061,76.120219,30,68,0.306122,0.693878
4,deep_false_trad_true,52,0.365385,0.807692,0.576923,150.230769,72.572673,26,26,0.500000,0.500000


In [69]:
len(deep_false_trad_true[deep_false_trad_true['true_label']== 'OFF'])
#len(deep_false_trad_true[deep_false_trad_true['true_label']== 'NOT'])

19

In [91]:
all_wrong.to_csv('comparison_gpt_trad_allwrong.csv', index=False)

In [92]:
all_correct.to_csv('comparison_gpt_trad_allcorrect.csv', index=False)

In [93]:
deep_true_trad_false.to_csv('comparison_gpt_true_trad_false.csv', index=False)
deep_false_trad_true.to_csv('comparison_gpt_false_trad_true.csv', index=False)